# Creating JSON Columns in House Value Dataset

We will be showing create a table in Db2 that utilizes JSON columns by using the JSON2BSON and BSON2JSON system calls.

### Import Python Modules

In some cases if your `import ibm_db` and `import ibm_db_dbi` does not work. Please refer to github issue [here](https://github.com/ibmdb/python-ibmdb/issues/276).

#### IMPORTANT NOTE: 
If you have a Mac, you will need to refer to the github link above and run the `install_name_tool`. Please make sure that you edit that line properly and make sure the paths are correct, otherwise it will affect other python dependencies!!!

In [ ]:
!easy_install ibm_db

In [1]:
import ibm_db
import ibm_db_dbi
import pandas as pd

### Connect to DB2 Instance
Here we will be connecting to our Db2 instance with the following service credentials. Please also enter the schema name you would like your table to be under.

In [4]:
# DON'T TOUCH THE `DRIVER` ATTRIBUTE! 
dsn = "DRIVER={{IBM DB2 ODBC DRIVER}};" + \
      "DATABASE={DATABASE_NAME};" + \
      "HOSTNAME={HOST_NAME};" + \
      "PORT=50000;" + \
      "PROTOCOL=TCPIP;" + \
      "UID={UID};" + \
      "PWD={PWD};"
SCHEMA_NAME = 'KXQ49540'
hdbc  = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)
print('Connection To DB2 Instance Has Been Created!')

Connection To DB2 Instance Has Been Created!


### Creat Table With JSON Columns
Here we will be creating a table for our House Value dataset. Notice how `BLDGTYPE` and  `HOUSESTYLE` have type `BLOB`. This indicates that those columns will be BSON columns. 

In [27]:
sql = 'CREATE TABLE '+SCHEMA_NAME+'.HOME_SALES ( ' + \
        'ID SMALLINT, ' + \
        'LOTAREA INTEGER, ' + \
        'BLDGTYPE BLOB,' + \
        'HOUSESTYLE BLOB, ' + \
        'OVERALLCOND INTEGER, ' + \
        'YEARBUILT INTEGER, ' + \
        'ROOFSTYLE VARCHAR(50), ' + \
        'EXTERCOND VARCHAR(50), ' + \
        'FOUNDATION VARCHAR(50), ' + \
        'BSMTCOND VARCHAR(50), ' + \
        'HEATING VARCHAR(50), ' + \
        'HEATINGQC VARCHAR(50),' + \
        'CENTRALAIR VARCHAR(50), ' + \
        'ELECTRICAL VARCHAR(50), ' + \
        'FULLBATH INTEGER, ' + \
        'HALFBATH INTEGER, ' + \
        'BEDROOMABVGR INTEGER, ' + \
        'KITCHENABVGR INTEGER, ' + \
        'KITCHENQUAL VARCHAR(50), ' + \
        'TOTRMSABVGRD INTEGER, ' + \
        'FIREPLACES INTEGER, ' + \
        'FIREPLACEQU VARCHAR(50), ' + \
        'GARAGETYPE VARCHAR(50), ' + \
        'GARAGEFINISH VARCHAR(50), ' + \
        'GARAGECARS INTEGER, ' + \
        'GARAGECOND VARCHAR(50), ' + \
        'POOLAREA INTEGER, ' + \
        'POOLQC VARCHAR(50), ' + \
        'FENCE VARCHAR(50), ' + \
        'MOSOLD INTEGER, ' + \
        'YRSOLD INTEGER, ' + \
        'SALEPRICE INTEGER )'


Time to execute our SQL statement and create a table. You may go to your DB2 instance and verify that the table has been created.

In [28]:
try:
    stmt = ibm_db.exec_immediate(hdbc, sql)
    print('Table HOME_SALES Has Been Created Under ' + str(SCHEMA_NAME) + ' Schema!')
except:
    print('ERROR: Table Already Exist Or Some Other Error')

ERROR: Table Already Exist Or Some Other Error


### Prepare and Load File into Database
As mentioned before, we will be using the home_sales dataset. We will be reading each row at a time since we need to convert two columns into a JSON before inserting into the db.

In [5]:
data = pd.read_csv("../data/home-sales-training-data.csv") 
data.head()

,ID,LOTAREA,BLDGTYPE,HOUSESTYLE,OVERALLCOND,YEARBUILT,ROOFSTYLE,EXTERCOND,FOUNDATION,BSMTCOND,...,GARAGETYPE,GARAGEFINISH,GARAGECARS,GARAGECOND,POOLAREA,POOLQC,FENCE,MOSOLD,YRSOLD,SALEPRICE
0,1,8450,1Fam,2Story,5,2003,Gable,TA,PConc,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,2,2008,208500
1,2,9600,1Fam,1Story,8,1976,Gable,TA,CBlock,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,5,2007,181500
2,3,11250,1Fam,2Story,5,2001,Gable,TA,PConc,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,9,2008,223500
3,4,9550,1Fam,2Story,5,1915,Gable,TA,BrkTil,Gd,...,Detchd,Unf,3,TA,0,NaN,NaN,2,2006,140000
4,5,14260,1Fam,2Story,5,2000,Gable,TA,PConc,TA,...,Attchd,RFn,3,TA,0,NaN,NaN,12,2008,250000


In [8]:
sql_p1 = 'INSERT INTO '+SCHEMA_NAME+'.HOME_SALES (' + \
        'ID, ' + \
        'LOTAREA, ' + \
        'BLDGTYPE,' + \
        'HOUSESTYLE, ' + \
        'OVERALLCOND, ' + \
        'YEARBUILT, ' + \
        'ROOFSTYLE, ' + \
        'EXTERCOND, ' + \
        'FOUNDATION, ' + \
        'BSMTCOND, ' + \
        'HEATING, ' + \
        'HEATINGQC,' + \
        'CENTRALAIR, ' + \
        'ELECTRICAL, ' + \
        'FULLBATH, ' + \
        'HALFBATH, ' + \
        'BEDROOMABVGR, ' + \
        'KITCHENABVGR, ' + \
        'KITCHENQUAL, ' + \
        'TOTRMSABVGRD, ' + \
        'FIREPLACES, ' + \
        'FIREPLACEQU, ' + \
        'GARAGETYPE, ' + \
        'GARAGEFINISH, ' + \
        'GARAGECARS , ' + \
        'GARAGECOND, ' + \
        'POOLAREA , ' + \
        'POOLQC, ' + \
        'FENCE, ' + \
        'MOSOLD, ' + \
        'YRSOLD, ' + \
        'SALEPRICE )'

As you can see, we are going through each row of the pandas dataframe and extracting each value. Notice how we are wrapper `BLDGTYPE` and `HOUSESTYLE` in a JSON object. We then put the JSON object in a system call function - `JSON2BSON`. This function converts the JSON object into a BSON, which is a binary representation of the JSON object. So when you view your data in the table, these two columns will be representated in binaray form. 

In [11]:
for index, row in data.iterrows():
    sql_p2 = ' VALUES ('+str(row['ID'])+' , ' + \
        ''+str(row['LOTAREA'])+' , ' + \
        'SYSTOOLS.JSON2BSON(\' {  "BLDGTYPE": "'+str(row['BLDGTYPE'])+'"}  \') , ' + \
        'SYSTOOLS.JSON2BSON(\' {  "HOUSESTYLE": "'+str(row['HOUSESTYLE'])+'"}  \') , ' + \
        ''+str(row['OVERALLCOND'])+' , ' + \
        ''+str(row['YEARBUILT'])+' , ' + \
        '\''+str(row['ROOFSTYLE'])+'\' , ' + \
        '\''+str(row['EXTERCOND'])+'\' , ' + \
        '\''+str(row['FOUNDATION'])+'\' , ' + \
        '\''+str(row['BSMTCOND'])+'\' , ' + \
        '\''+str(row['HEATING'])+'\' , ' + \
        '\''+str(row['HEATINGQC'])+'\' , ' + \
        '\''+str(row['CENTRALAIR'])+'\' , ' + \
        '\''+str(row['ELECTRICAL'])+'\' , ' + \
        ''+str(row['FULLBATH'])+' , ' + \
        ''+str(row['HALFBATH'])+' , ' + \
        ''+str(row['BEDROOMABVGR'])+' , ' + \
        ''+str(row['KITCHENABVGR'])+' , ' + \
        '\''+str(row['KITCHENQUAL'])+'\' , ' + \
        ''+str(row['TOTRMSABVGRD'])+' , ' + \
        ''+str(row['FIREPLACES'])+' , ' + \
        '\''+str(row['FIREPLACEQU'])+'\' , ' + \
        '\''+str(row['GARAGETYPE'])+'\' , ' + \
        '\''+str(row['GARAGEFINISH'])+'\' , ' + \
        ''+str(row['GARAGECARS'])+' , ' + \
        '\''+str(row['GARAGECOND'])+'\' , ' + \
        ''+str(row['POOLAREA'])+' , ' + \
        '\''+str(row['POOLQC'])+'\' , ' + \
        '\''+str(row['FENCE'])+'\' , ' + \
        ''+str(row['MOSOLD'])+' , ' + \
        ''+str(row['YRSOLD'])+' , ' + \
        ''+str(row['SALEPRICE'])+' ' + \
        ')'
    sql_final  = sql_p1 + sql_p2
    stmt = ibm_db.exec_immediate(hdbc, sql_final)

# Viewing and Pulling Data From Database
Now that we have inserted all our data into our database, we want to be able to read and extract information. For that we will use the `SELECT` statement. Notice that we will be using the system call `BSON2JSON`. Since two of your columns are JSON objects and stored in the database as binary objects, we need to convert them back to JSON so that we can read and use that data effectively. 

In [14]:
sql = 'SELECT LOTAREA, MOSOLD, YRSOLD, SALEPRICE, SYSTOOLS.BSON2JSON(BLDGTYPE) AS BLDGTYPE , SYSTOOLS.BSON2JSON(HOUSESTYLE) AS HOUSESTYLE FROM '+SCHEMA_NAME+'.HOME_SALES;'

In [15]:
df = pd.read_sql(sql,hdbi)
df.head()

,LOTAREA,MOSOLD,YRSOLD,SALEPRICE,BLDGTYPE,HOUSESTYLE
0,8450,2,2008,208500,"{""BLDGTYPE"":""1Fam""}","{""HOUSESTYLE"":""2Story""}"
1,9600,5,2007,181500,"{""BLDGTYPE"":""1Fam""}","{""HOUSESTYLE"":""1Story""}"
2,11250,9,2008,223500,"{""BLDGTYPE"":""1Fam""}","{""HOUSESTYLE"":""2Story""}"
3,9550,2,2006,140000,"{""BLDGTYPE"":""1Fam""}","{""HOUSESTYLE"":""2Story""}"
4,14260,12,2008,250000,"{""BLDGTYPE"":""1Fam""}","{""HOUSESTYLE"":""2Story""}"


In [17]:
sql = 'SELECT LOTAREA, MOSOLD, YRSOLD, SALEPRICE, JSON_VAL(BLDGTYPE,\'BLDGTYPE\',\'s:36\') AS BLDGTYPE , JSON_VAL(HOUSESTYLE,\'HOUSESTYLE\',\'s:36\') AS HOUSESTYLE FROM '+SCHEMA_NAME+'.HOME_SALES;'

In [18]:
df = pd.read_sql(sql,hdbi)
df.head()

,LOTAREA,MOSOLD,YRSOLD,SALEPRICE,BLDGTYPE,HOUSESTYLE
0,8450,2,2008,208500,1Fam,2Story
1,9600,5,2007,181500,1Fam,1Story
2,11250,9,2008,223500,1Fam,2Story
3,9550,2,2006,140000,1Fam,2Story
4,14260,12,2008,250000,1Fam,2Story
